In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver.implicitly_wait(5)
# s_data = []

# driver.get('https://sg.sbiz.or.kr/godo/stat/upso.sg')
# driver.find_element(By.CSS_SELECTOR,'a#twoTypePopup').click()
# time.sleep(1)
# si = bizinfo_df['address'][0][:2]
# driver.find_element(By.XPATH, f"//label[text()='{si}']").click()   #(By.XPATH, "//label[text()='"+si+"']").click()

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(3)
s_data, 행정구역, 행정구역, 업종, 주간인구수, 밀집도, 밀집도_중위값, 밀집정도, 동종근로자_임금_수준미만, 동종근로자_임금_수준미만_척도 = [],[],[],[],[],[],[],[],[],[] 

driver.get('https://sg.sbiz.or.kr/godo/stat/updens.sg')
for region in range(1,18):
    for industry in range(2,23):
        driver.find_element(By.XPATH,'//*[@id="page4-1"]/div[1]/div/div[1]/ul/li[1]/div').click()
        time.sleep(2)
        driver.execute_script("window.scrollTo(0,100)")
        driver.find_element(By.XPATH, f"//label[@for='select_11_{region}']").click()   
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="ctyUl"]/li[1]/label').click()
        driver.find_element(By.XPATH, '//*[@id="popupbox"]/div/div[2]/div/div[2]/a[2]').click()  
        driver.find_element(By.XPATH,'//*[@id="page4-1"]/div[1]/div/div[1]/ul/li[2]/div').click()
        time.sleep(2)
        driver.execute_script("window.scrollTo(0,200)")
        driver.find_element(By.XPATH, f'//*[@id="upjong1Ul"]/li[{industry}]/label').click()
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="upjong2Ul"]/li[1]/label').click()
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="popupbox"]/div/div[2]/div/div[2]/a[2]').click()
        driver.find_element(By.XPATH, '//*[@id="page4-1"]/div[1]/div/div[2]/a').click()
        time.sleep(2)
        page_source = driver.page_source
        soup = bs(page_source, 'lxml')
        try:
            soup.select('#updensTable tr')[1]
        except: pass
        else:
            table = soup.select('#updensTable td')
            for i in range(len(table)//9):
                행정구역 = [table[9*i].text]
                업종 = [table[9*i+1].text]
                주간인구수 = [table[9*i+2].text]
                밀집도 = [table[9*i+3].text]
                밀집도_중위값 = [table[9*i+4].text]
                밀집정도 = [table[9*i+5].text]
                동종근로자_임금_수준미만 = [table[9*i+6].text]
                동종근로자_임금_수준미만_척도 = [table[9*i+7].text]
                s_data.append([행정구역, 업종, 주간인구수, 밀집도, 밀집도_중위값, 밀집정도, 동종근로자_임금_수준미만, 동종근로자_임금_수준미만_척도])

In [ ]:
sbiz_df = pd.DataFrame({'행정구역': s_data[i][0], '업종': s_data[i][1], '주간인구수': s_data[i][2], 
                           '밀집도' : s_data[i][3], '밀집도_중위값' : s_data[i][4], '밀집정도' : s_data[i][5], 
                           '동종근로자_임금_수준미만' : s_data[i][6], '동종근로자_임금_수준미만_척도': s_data[i][7]} for i in range(len(행정구역)))
sbiz_df.to_csv('sbiz_df.csv') 